<a href="https://colab.research.google.com/github/diviramon/BASSic/blob/master/PanoGAN_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Thu Mar  4 20:16:57 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.39       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    24W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install pydicom
!pip install pytorch-msssim

     |████████████████████████████████| 1.9MB 13.5MB/s 


# Import Libraries

In [1]:
DIM = 64
OUTPUT_DIM = DIM*DIM*1

import shutil
import os
import cv2
import re
import random
from skimage.io import imread 
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import random
import pandas as pd
from torch import nn
from torch.autograd import grad
import torch
from torchvision import datasets, transforms
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.data import random_split, ConcatDataset

from collections import OrderedDict
import time
from tqdm import tqdm

from torch.autograd import Variable
import torch.optim as optim
import torch.utils.data
import torchvision.utils as vutils

from sklearn.metrics import roc_curve, auc, average_precision_score, f1_score
from scipy.optimize import brentq
from scipy.interpolate import interp1d
import matplotlib.pyplot as plt
from matplotlib import rc
from google.colab.patches import cv2_imshow
import pydicom as dcm
from skimage import io, exposure
from pytorch_msssim import ssim, ms_ssim, SSIM, MS_SSIM

from torch.utils.data import DataLoader
from torch import autograd
from torchvision.utils import save_image
from torch.utils.data import sampler
from argparse import ArgumentParser
from sklearn import metrics

# Make Folders

In [5]:
!rm -r sample_data/
!mkdir data
!mkdir data/train
!mkdir data/train/healthy
!mkdir data/valid
!mkdir data/valid/healthy
!mkdir data/valid/nodule
!mkdir data/test/
!mkdir data/test/healthy
!mkdir data/test/nodule
!mkdir data/patients/
!mkdir output/
!mkdir output/patients/

!mkdir encoder

# Read Image Functions

In [3]:
def read_indiana(file_name, cxr_path, mask_path):
  
  dcm_filename = file_name[3:-4] + '.dcm'

  # Pre-process mask
  mask = cv2.imread(os.path.join(mask_path, file_name))
  mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
  mask = cv2.resize(mask, (1024, 1024), interpolation=cv2.INTER_NEAREST)
  mask = (mask > 127) * 255

  # Read in DICOM 
  ds = dcm.dcmread(os.path.join(cxr_path, dcm_filename))
  cxr = ds.pixel_array
  #print(ds.PhotometricInterpretation)
  #cxr = dcm.pixel_data_handlers.util.apply_voi_lut(cxr, ds, index=0)
  cxr = cv2.resize(cxr, (1024, 1024))
  if ds.PhotometricInterpretation == 'MONOCHROME1':
    cxr = 1.0 - cxr * 1./cxr.max()
  else:
    cxr = cxr * 1. / cxr.max()
  cxr = exposure.equalize_adapthist(cxr,kernel_size=128)
  normalizedImg = np.zeros((1024, 1024))
  cxr = cv2.normalize(cxr,  normalizedImg, 0, 255, cv2.NORM_MINMAX)

  return mask, cxr

In [8]:
def read_nih(file_name, cxr_path, mask_path):
  
  # Pre-process mask
  mask = cv2.imread(os.path.join(mask_path, file_name))
  mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
  mask = cv2.resize(mask, (1024, 1024), interpolation=cv2.INTER_NEAREST)
  mask = (mask > 127) * 255
  
  # Read in PNG
  cxr = cv2.imread(os.path.join(cxr_path, file_name))
  cxr = cv2.cvtColor(cxr, cv2.COLOR_BGR2GRAY)
  cxr = cv2.resize(cxr, (1024, 1024)) 
  cxr = cxr * 1./cxr.max()
  cxr = exposure.equalize_adapthist(cxr,kernel_size=128)
  normalizedImg = np.zeros((1024, 1024))
  cxr = cv2.normalize(cxr,  normalizedImg, 0, 255, cv2.NORM_MINMAX)

  return mask, cxr

In [9]:
def read_jsrt(file_name, cxr_path, mask_path):

  # Pre-process mask
  mask = cv2.imread(os.path.join(mask_path, file_name))
  mask = cv2.cvtColor(mask, cv2.COLOR_BGR2GRAY)
  mask = cv2.resize(mask, (1024, 1024), interpolation=cv2.INTER_NEAREST)
  mask = (mask > 127) * 255

  # Read in IMG
  # fname = file_name[:-4] + '.IMG'
  # cxr = 1.0 - np.fromfile(os.path.join(cxr_path, fname), dtype='>u2').reshape((2048, 2048)) * 1. / 4095
  cxr = cv2.imread(os.path.join(cxr_path, file_name))
  cxr = cv2.cvtColor(cxr, cv2.COLOR_BGR2GRAY)
  cxr = cv2.resize(cxr, (1024, 1024)) 
  cxr = cxr * 1./cxr.max()
  cxr = exposure.equalize_adapthist(cxr,kernel_size=128)
  normalizedImg = np.zeros((1024, 1024))
  cxr = cv2.normalize(cxr,  normalizedImg, 0, 255, cv2.NORM_MINMAX)

  return mask, cxr

In [10]:
def crop_patches(image, mask, total_patches, box, image_name, dataset, svpath):
  if dataset == 'indiana':
    patch_extractor = ExtractPatches(image_name, image, mask, box, DIM, DIM//2, dataset, svpath)
  else:
    patch_extractor = ExtractPatches(image_name, image, mask, box, DIM, DIM//4, dataset, svpath)
  
  lung_area = np.count_nonzero(mask)
  patches = patch_extractor.extract_all_patches()
  total_patches = total_patches + patches

  return total_patches

In [11]:
class ExtractPatches:
    def __init__(self, image_name, image, mask, bbox, patchSize, stride, dataset, out_path):
        self.image = image
        self.out_path = out_path
        self.dataset = dataset
        self.bbox = bbox
        self.annotated = np.copy(image) 
        self.reconstructed = np.empty(image.shape)
        self.reconstructed[:] = np.nan
        self.image_name = image_name
        self.patchSize = patchSize
        self.stride = stride
        self.mask = mask

    def draw_single_patches(self, coords, patch):
      y1 = coords[0] * self.stride
      x1 = coords[1] * self.stride
      y2 = coords[0] * self.stride + self.patchSize
      x2 = coords[1] * self.stride + self.patchSize
      self.annotated = cv2.rectangle(self.annotated,(x1,y1), (x2,y2), (0, 0, 255))
      self.reconstructed[y1:y2, x1:x2] = patch
    
    def extract_single_patches(self, patch):
        croppedPatches = self.image[(patch[0] * self.stride):(patch[0] * self.stride + self.patchSize), 
                               (patch[1] * self.stride):(patch[1] * self.stride + self.patchSize)]
        return croppedPatches

    def no_of_patches(self):
        yNoOfPatches, xNoOfPatches = (int((self.image.shape[1] - self.patchSize) / self.stride + 1),
                                      int((self.image.shape[0] - self.patchSize) / self.stride + 1))
        return xNoOfPatches, yNoOfPatches

    def extract_all_patches(self):
        xNoOfPatches, yNoOfPatches = self.no_of_patches()
        closest = None
        closest_dist = 10000

        allPatches = list()
        for y in range(yNoOfPatches):
            for x in range(xNoOfPatches):
              patch = self.extract_single_patches((x,y))
                            
              y1 = x * self.stride
              x1 = y * self.stride
              y2 = x * self.stride + self.patchSize
              x2 = y * self.stride + self.patchSize
              mask_cover = np.sum(self.mask[y1:y2, x1:x2])
              if x1 > 0 and x2 < 1024 and y1 > 0 and y2 < 1024:
                if patch.shape[0] == self.patchSize and patch.shape[1] and mask_cover/(255*self.patchSize*self.patchSize) > 0.75:
                  if self.dataset == 'jsrt-nodule':
                      cx = self.bbox[0]
                      cy = self.bbox[1]
                      if (x1 < cx) and (x2 > cx) and (y1 < cy) and (y2 > cy):
                          bbox_cx = (x1+x2)//2
                          bbox_cy = (y1+y2)//2
                          if ((bbox_cx-cx)**2 + (bbox_cy-cy)**2) < closest_dist:
                              closest_dist = (bbox_cx-cx)**2 + (bbox_cy-cy)**2
                              closest = patch
                              closest_x = x
                              closest_y = y
                          self.draw_single_patches((x,y), patch)
                  elif self.dataset == 'nih-nodule':
                      cx = (self.bbox[0] + self.bbox[2])//2
                      cy = (self.bbox[1] + self.bbox[3])//2
                      if (x1 < cx) and (x2 > cx) and (y1 < cy) and (y2 > cy):
                          bbox_cx = (x1+x2)//2
                          bbox_cy = (y1+y2)//2
                          if ((bbox_cx-cx)**2 + (bbox_cy-cy)**2) < closest_dist:
                              closest_dist = (bbox_cx-cx)**2 + (bbox_cy-cy)**2
                              closest = patch
                              closest_x = x
                              closest_y = y
                          self.draw_single_patches((x,y), patch)
                  else:
                      allPatches.append(patch)
                      cv2.imwrite(self.out_path + str(x) + '_' + str(y) + '_' + self.image_name, patch)
                      # plt.imsave(self.out_path + str(x) + '_' + str(y) + '_' + self.image_name, patch, cmap='gray')
                      self.draw_single_patches((x,y), patch)
        
        if closest is not None:
            allPatches.append(closest)
            cv2.imwrite(self.out_path + str(closest_x) + '_' + str(closest_y) + '_' + self.image_name, closest)
            # plt.imsave(self.out_path + str(closest_x) + '_' + str(closest_y) + '_' + self.image_name, closest, cmap='gray')

        #cv2_imshow(self.annotated)

        return allPatches

# Paths

In [2]:
jsrt_mask_path = '/content/drive/MyDrive/FYDP 2021/JSRT Data/JSRT Masks'
jsrt_cxr_path = '/content/drive/MyDrive/FYDP 2021/JSRT Data/JSRT Images'

nih_healthy_mask_path = '/content/drive/MyDrive/FYDP 2021/NIH-Data/Healthy_masks'
nih_healthy_cxr_path = '/content/drive/MyDrive/FYDP 2021/NIH-Data/Healthy_raw'

nih_nodule_mask_path = '/content/drive/MyDrive/FYDP 2021/NIH-Data/Nodule_masks'
nih_nodule_cxr_path = '/content/drive/MyDrive/FYDP 2021/NIH-Data/Nodule_raw'

indiana_mask_path = '/content/drive/My Drive/FYDP 2021/Indiana University Database/Additional Masks'
indiana_cxr_path = '/content/drive/My Drive/FYDP 2021/Indiana University Database/DICOM Frontal'

nih_healthy_img_list = os.listdir(nih_healthy_mask_path)
indiana_img_list = os.listdir(indiana_mask_path)

nih_info = pd.read_csv('/content/drive/MyDrive/FYDP 2021/NIH-Data/BBox_List_2017.csv')
nih_info = nih_info[nih_info['Finding Label']=='Nodule']
nih_nodule_list = nih_info['Image Index'].values

jsrt_info = pd.read_csv('/content/drive/MyDrive/FYDP 2021/JSRT Data/jsrt_metadata (1).csv')
jsrt_healthy_info = jsrt_info[jsrt_info['state']=='non-nodule']
jsrt_healthy_list = jsrt_healthy_info['study_id'].values
jsrt_nodule_info = jsrt_info[jsrt_info['state']!='non-nodule']
jsrt_nodule_list = jsrt_nodule_info['study_id'].values

batchsize = 64
random.seed(42)
np.random.seed(42)

# Load Images

### Indiana

In [13]:
total_patches = []
for j in range(len(indiana_img_list)):
    image_name = indiana_img_list[j]
    #print(image_name)
    box = []
    mask, cxr = read_indiana(image_name, indiana_cxr_path, indiana_mask_path)
    total_patches = crop_patches(cxr, mask, total_patches, box, image_name, 'indiana', 'data/train/healthy/')
len(total_patches)

70798

### NIH

In [14]:
total_patches = []
for j in range(len(nih_healthy_img_list)):
    image_name = nih_healthy_img_list[j]
    #print(image_name)
    box = []
    mask, cxr = read_nih(image_name, nih_healthy_cxr_path, nih_healthy_mask_path)
    total_patches = crop_patches(cxr, mask, total_patches, box, image_name, 'nih-healthy', 'data/valid/healthy/')
len(total_patches)

9010

In [15]:
total_patches = []
for j in range(len(nih_nodule_list)):
    image_name = nih_nodule_list[j]
    #print(image_name)
    bx = nih_info['Bbox [x'].values[j]
    by = nih_info['y'].values[j]
    bh = nih_info['h]'].values[j]
    bw = nih_info['w'].values[j]
    box = [bx,by, bx+bw, by+bh]
    mask, cxr = read_nih(image_name, nih_nodule_cxr_path, nih_nodule_mask_path)
    total_patches = crop_patches(cxr, mask, total_patches, box, image_name, 'nih-nodule', 'data/valid/nodule/')
len(total_patches)

73

In [16]:
files = os.listdir(path='data/valid/healthy')
nfiles = len(files) + len(os.listdir("data/valid/nodule"))
if nfiles % batchsize != 0:
  del_files = random.sample(files,nfiles%batchsize)
  for dfile in del_files:
    os.remove(os.path.join("data/valid/healthy/", dfile))
  print("Files removed: ", len(del_files))

Files removed:  59


## JSRT

In [ ]:
total_patches = []
for j in range(len(jsrt_nodule_list)):
    image_name = jsrt_nodule_list[j]
    #print(image_name)
    bx = jsrt_nodule_info['x'].values[j]/2
    by = jsrt_nodule_info['y'].values[j]/2
    box = [bx,by]
    mask, cxr = read_jsrt(image_name, jsrt_cxr_path, jsrt_mask_path)
    total_patches = crop_patches(cxr, mask, total_patches, box, image_name, 'jsrt-nodule', 'data/test/nodule/')
len(total_patches)

In [ ]:
total_patches = []
jsrt_healthy_list = list(jsrt_healthy_list)

for j in range(len(jsrt_healthy_list)):
    image_name = jsrt_healthy_list[j]
    print(image_name)
    box = []
    mask, cxr = read_jsrt(image_name, jsrt_cxr_path, jsrt_mask_path)
    total_patches = crop_patches(cxr, mask, total_patches, box, image_name, 'jsrt-healthy', 'data/test/healthy/')
len(total_patches)

In [ ]:
files = os.listdir(path='data/test/healthy')
nfiles = len(files) + len(os.listdir("data/test/nodule"))
if nfiles % batchsize != 0:
  del_files = random.sample(files,nfiles%batchsize)
  for dfile in del_files:
    os.remove(os.path.join("data/test/healthy/", dfile))
  print("Files removed: ", len(del_files))

# NN Functions

In [5]:
class MyConvo2d(nn.Module):
    def __init__(self, input_dim, output_dim, kernel_size, he_init=True,  stride=1, bias=True):
        super(MyConvo2d, self).__init__()
        self.he_init = he_init
        self.padding = int((kernel_size - 1)/2)
        self.conv = nn.Conv2d(input_dim, output_dim, kernel_size,
                              stride=1, padding=self.padding, bias=bias)

    def forward(self, input):
        output = self.conv(input)
        return output

In [6]:
class ConvMeanPool(nn.Module):
    def __init__(self, input_dim, output_dim, kernel_size, he_init=True):
        super(ConvMeanPool, self).__init__()
        self.he_init = he_init
        self.conv = MyConvo2d(input_dim, output_dim,
                              kernel_size, he_init=self.he_init)

    def forward(self, input):
        output = self.conv(input)
        output = (output[:, :, ::2, ::2] + output[:, :, 1::2, ::2] +
                  output[:, :, ::2, 1::2] + output[:, :, 1::2, 1::2]) / 4
        return output

In [7]:
class MeanPoolConv(nn.Module):
    def __init__(self, input_dim, output_dim, kernel_size, he_init=True):
        super(MeanPoolConv, self).__init__()
        self.he_init = he_init
        self.conv = MyConvo2d(input_dim, output_dim,
                              kernel_size, he_init=self.he_init)

    def forward(self, input):
        output = input
        output = (output[:, :, ::2, ::2] + output[:, :, 1::2, ::2] +
                  output[:, :, ::2, 1::2] + output[:, :, 1::2, 1::2]) / 4
        output = self.conv(output)
        return output

In [8]:
class DepthToSpace(nn.Module):
    def __init__(self, block_size):
        super(DepthToSpace, self).__init__()
        self.block_size = block_size
        self.block_size_sq = block_size*block_size

    def forward(self, input):
        output = input.permute(0, 2, 3, 1)
        (batch_size, input_height, input_width, input_depth) = output.size()
        output_depth = int(input_depth / self.block_size_sq)
        output_width = int(input_width * self.block_size)
        output_height = int(input_height * self.block_size)
        t_1 = output.reshape(batch_size, input_height,
                             input_width, self.block_size_sq, output_depth)
        spl = t_1.split(self.block_size, 3)
        stacks = [t_t.reshape(batch_size, input_height,
                              output_width, output_depth) for t_t in spl]
        output = torch.stack(stacks, 0).transpose(0, 1).permute(0, 2, 1, 3, 4).reshape(
            batch_size, output_height, output_width, output_depth)
        output = output.permute(0, 3, 1, 2)
        return output

In [9]:
class UpSampleConv(nn.Module):
    def __init__(self, input_dim, output_dim, kernel_size, he_init=True, bias=True):
        super(UpSampleConv, self).__init__()
        self.he_init = he_init
        self.conv = MyConvo2d(input_dim, output_dim,
                              kernel_size, he_init=self.he_init, bias=bias)
        self.depth_to_space = DepthToSpace(2)

    def forward(self, input):
        output = input
        output = torch.cat((output, output, output, output), 1)
        output = self.depth_to_space(output)
        output = self.conv(output)
        return output

In [10]:
class ResidualBlock(nn.Module):
    def __init__(self, input_dim, output_dim, kernel_size, resample=None, hw=DIM):
        super(ResidualBlock, self).__init__()

        self.input_dim = input_dim
        self.output_dim = output_dim
        self.kernel_size = kernel_size
        self.resample = resample
        self.bn1 = None
        self.bn2 = None
        self.relu1 = nn.ReLU()
        self.relu2 = nn.ReLU()
        if resample == 'down':
            self.bn1 = nn.LayerNorm([input_dim, hw, hw])
            self.bn2 = nn.LayerNorm([input_dim, hw, hw])
        elif resample == 'up':
            self.bn1 = nn.BatchNorm2d(input_dim)
            self.bn2 = nn.BatchNorm2d(output_dim)
        elif resample == 'custom':
            self.bn1 = nn.InstanceNorm2d(input_dim)
            self.bn2 = nn.InstanceNorm2d(output_dim)
        else:
            raise Exception('invalid resample value')

        if resample == 'down':
            self.conv_shortcut = MeanPoolConv(
                input_dim, output_dim, kernel_size=1, he_init=False)
            self.conv_1 = MyConvo2d(
                input_dim, input_dim, kernel_size=kernel_size, bias=False)
            self.conv_2 = ConvMeanPool(
                input_dim, output_dim, kernel_size=kernel_size)
        elif resample == 'up':
            self.conv_shortcut = UpSampleConv(
                input_dim, output_dim, kernel_size=1, he_init=False)
            self.conv_1 = UpSampleConv(
                input_dim, output_dim, kernel_size=kernel_size, bias=False)
            self.conv_2 = MyConvo2d(
                output_dim, output_dim, kernel_size=kernel_size)
        elif resample == "custom":
            self.conv_shortcut = MeanPoolConv(
                input_dim, output_dim, kernel_size=1, he_init=False)
            self.conv_1 = MyConvo2d(
                input_dim, input_dim, kernel_size=kernel_size, bias=False)
            self.conv_2 = ConvMeanPool(
                input_dim, output_dim, kernel_size=kernel_size)
        else:
            raise Exception('invalid resample value')

    def forward(self, input):
        if self.input_dim == self.output_dim and self.resample == None:
            shortcut = input
        else:
            shortcut = self.conv_shortcut(input)

        output = input
        output = self.bn1(output)
        output = self.relu1(output)
        output = self.conv_1(output)
        output = self.bn2(output)
        output = self.relu2(output)
        output = self.conv_2(output)

        return shortcut + output

In [11]:
class ReLULayer(nn.Module):
    def __init__(self, n_in, n_out):
        super(ReLULayer, self).__init__()
        self.n_in = n_in
        self.n_out = n_out
        self.linear = nn.Linear(n_in, n_out)
        self.relu = nn.ReLU()

    def forward(self, input):
        output = self.linear(input)
        output = self.relu(output)
        return output

In [12]:
class FCGenerator(nn.Module):
    def __init__(self, FC_DIM=512):
        super(FCGenerator, self).__init__()
        self.relulayer1 = ReLULayer(128, FC_DIM)
        self.relulayer2 = ReLULayer(FC_DIM, FC_DIM)
        self.relulayer3 = ReLULayer(FC_DIM, FC_DIM)
        self.relulayer4 = ReLULayer(FC_DIM, FC_DIM)
        self.linear = nn.Linear(FC_DIM, OUTPUT_DIM)
        self.tanh = nn.Tanh()

    def forward(self, input):
        output = self.relulayer1(input)
        output = self.relulayer2(output)
        output = self.relulayer3(output)
        output = self.relulayer4(output)
        output = self.linear(output)
        output = self.tanh(output)
        return output

In [13]:
class Encoder(nn.Module):
    def __init__(self, dim, output_dim, drop_rate=0.0):
        super(Encoder, self).__init__()
        self.dropout = nn.Dropout(drop_rate)
        self.conv_in = nn.Conv2d(1, dim, 3, 1, padding=1)
        self.res1 = ResidualBlock(dim, dim*2, 3, 'down', DIM)
        self.res2 = ResidualBlock(dim*2, dim*4, 3, 'down', int(DIM/2))
        self.res3 = ResidualBlock(dim*4, dim*8, 3, 'down', int(DIM/4))
        self.res4 = ResidualBlock(dim*8, dim*8, 3, 'down', int(DIM/8))
        self.fc = nn.Linear(4*4*8*dim, output_dim)

    def forward(self, x):
        # x = self.dropout(x)
        x = self.conv_in(x)
        x = self.res1(x)
        x = self.res2(x)
        x = self.res3(x)
        x = self.res4(x)
        x = x.view(x.size(0), -1)
        x = self.dropout(self.fc(x))
        return torch.tanh(x)

In [14]:
from pytorch_msssim import ssim

def ssim_loss(input,target,size_average=True):
    input = (input + 1) / 2
    target = (target + 1) / 2
    ssim_loss = 1 - ssim(input, target, data_range=1, size_average=size_average)
    return ssim_loss

# Flags

In [15]:
import sys
sys.path.append(os.getcwd())

device = torch.device('cuda:{}'.format(0))
torch.cuda.set_device('cuda:{}'.format(0))
MODE = 'wgan-gp'  # Valid options are dcgan, wgan, or wgan-gp
DIM = 64  # This overfits substantially; you're probably better off with 64
LAMBDA = 10  # Gradient penalty lambda hyperparameter
CRITIC_ITERS = 5  # How many critic iterations per generator iteration
BATCH_SIZE = batchsize  # Batch size
ITERS = 100000  # How many generator iterations to train for
OUTPUT_DIM = 1 * DIM * DIM  # Number of pixels in image (3*64*64)
NOISE_SIZE = 512

train_data_path = "data/train/"
valid_data_path = "data/valid/"
test_data_path = "data/test/"

torch.manual_seed(0)
np.random.seed(0)
torch.backends.cudnn.benchmark = True
torch.backends.cudnn.determinstic = False
torch.set_deterministic(True)

In [16]:
def calc_gradient_penalty(netD, real_data, fake_data):
    alpha = torch.rand(BATCH_SIZE, 1)
    alpha = alpha.expand(BATCH_SIZE, int(real_data.nelement()/BATCH_SIZE)).contiguous()
    alpha = alpha.view(BATCH_SIZE, 1, DIM, DIM)
    alpha = alpha.to(device)

    fake_data = fake_data.view(BATCH_SIZE, 1, DIM, DIM)
    interpolates = alpha * real_data.detach() + ((1 - alpha) * fake_data.detach())

    interpolates = interpolates.to(device)
    interpolates.requires_grad_(True)

    disc_interpolates = netD(interpolates)

    gradients = autograd.grad(outputs=disc_interpolates, inputs=interpolates,
                              grad_outputs=torch.ones(
                                  disc_interpolates.size()).to(device),
                              create_graph=True, retain_graph=True, only_inputs=True)[0]

    gradients = gradients.view(gradients.size(0), -1)
    gradient_penalty = ((gradients.norm(2, dim=1) - 1) ** 2).mean() * LAMBDA
    return gradient_penalty

# DataLoaders

In [17]:
def get_dataloaders(train_data_path, test_data_path, bs=batchsize):
    train_transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
        transforms.Grayscale(num_output_channels=1),
        transforms.RandomAffine(5, translate=[0.05,0.05]),
        transforms.RandomHorizontalFlip(),
        transforms.RandomVerticalFlip(),
    ])

    test_transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
        transforms.Grayscale(num_output_channels=1),
    ])
    
    trn_dataset = datasets.ImageFolder(root=train_data_path, transform=test_transform)
    tst_dataset = datasets.ImageFolder(root=test_data_path, transform=test_transform)

    trn_loader = torch.utils.data.DataLoader(trn_dataset, batch_size=bs, shuffle=True, drop_last=True)
    tst_loader = torch.utils.data.DataLoader(tst_dataset, batch_size=bs, shuffle=False)
  
    return trn_loader, tst_loader

# PGAN

In [18]:
!git clone https://nitishbhatt56:@github.com/nitishbhatt56/pytorch_GAN_zoo.git

fatal: destination path 'pytorch_GAN_zoo' already exists and is not an empty directory.


In [19]:
!unzip "/content/drive/MyDrive/FYDP 2021/PGAN_p64_s4_96k.zip"

Archive:  /content/drive/MyDrive/FYDP 2021/PGAN_p64_s4_96k.zip
replace output_networks/default/default_s3_i80000.pt? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [20]:
import pytorch_GAN_zoo.models.utils.utils as utils
import json
import sys
sys.path.insert(0,'./pytorch_GAN_zoo')

In [21]:
checkpt_dir = 'output_networks'
name='default'
checkPointDir = os.path.join(checkpt_dir, name)
module = 'PGAN'

checkpointData = utils.getLastCheckPoint(
            checkPointDir, name, scale=None, iter=None)

print(checkpointData)
modelConfig, pathModel, _ = checkpointData
with open(modelConfig, 'rb') as file:
    configData = json.load(file)

modelPackage, modelName = utils.getNameAndPackage(module)
modelType = utils.loadmodule(modelPackage, modelName)

model = modelType(useGPU=True,
                  storeAVG=True,
                  **configData)


model.load(pathModel)

('output_networks/default/default_train_config.json', 'output_networks/default/default_s4_i96000.pt', 'output_networks/default/default_s4_i96000_tmp_config.json')
Average network found !


# Encoder Training Function

In [22]:
def calc_alpha(e):
  if e < 75:
      return 0.1
  return 1

In [23]:
def train_encoder():
    netG = model.avgG
    netG.eval()
    netD = model.netD
    netD.eval()
    for p in netD.parameters():
        p.requires_grad = False
    for p in netG.parameters():
        p.requires_grad = False

    dataloader, validloader = get_dataloaders(train_data_path, valid_data_path, BATCH_SIZE)

    netE = Encoder(DIM,NOISE_SIZE,0.3).to(device)
    optimizer = optim.Adam(netE.parameters(), lr=1e-4, betas=(0.9, 0.999))
    crit = nn.MSELoss()
    rec_crit = nn.L1Loss()
    
    chkpts = os.listdir('encoder/')
    if not chkpts: 
      print('starting from scratch...')
      start_iter = 1
    else:
      print('loading checkpoint...')
      chkpts = sorted(chkpts, reverse=True, key=lambda x : float(re.split('_|p',x)[1]))
      checkpoint_fpathE = os.path.join('encoder/', chkpts[0])
      print(checkpoint_fpathE)
      checkpointEl = torch.load(checkpoint_fpathE)

      netE.load_state_dict(checkpointEl['state_dict'])
      optimizer.load_state_dict(checkpointEl['optimizer'])

      start_iter = checkpointEl['epoch']

    for e in range(start_iter, 300):
        losses = []
        netE.train()
        options_alpha = calc_alpha(e)
        for (x, _) in dataloader:
            x = x.to(device)
            code = netE(x)
            rec_image = netG(code)
            code_rec = netE(rec_image)
            f_x = netD.forward(x,getFeature=True)[1].detach()
            f_gx = netD.forward(rec_image,getFeature=True)[1]
            loss = ssim_loss(rec_image, x) + rec_crit(code_rec, code) + options_alpha * rec_crit(f_gx, f_x)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            losses.append(loss.item())

        print(e, np.mean(losses))
        netE.eval()
        rec_image = netG(netE(x))
        d_input = torch.cat((x, rec_image), dim=0)
        save_image(d_input, 'rec'+str(e)+'.jpg',normalize=True)
        if e % 1 == 0:
            checkpointE = {
              'epoch': e + 1,
              'state_dict': netE.state_dict(),
              'optimizer': optimizer.state_dict()
            }
            torch.save(checkpointE, 'encoder/netE_%d.pth' % e)
            evaluate(netG, netD, netE,options_alpha,validloader)
            shutil.copy('encoder/netE_%d.pth' % e,'/content/drive/MyDrive/FYDP 2021/ANOGAN-Weights/PanoGAN/encoder/running/netE_%d.pth' % e)
    torch.save(netE.state_dict(), 'wgangp/netE.pth')

# Evaluate Function

In [24]:
def evaluate(netG, netD, netE,options_alpha, dataloader):
    options_c = 0
    netE.eval()
    netD.eval()
    netE.eval()

    # crit = nn.MSELoss()
    y_true, y_score = [], []
    rec_scores, enc_scores, feat_scores = [], [], []
    in_real, out_real, in_rec, out_rec = [], [], [], []
    
    with torch.no_grad():
        for (x, label) in dataloader:
            bs = x.size(0)
            x = x.to(device)
            code = netE(x)
            rec_image = netG(code)
            rec_code = netE(rec_image)

            d_input = torch.cat((x, rec_image), dim=0)
            idx = (label == options_c)
            in_real.append(x[idx])
            in_rec.append(rec_image[idx])
            idx = (label != options_c)
            out_real.append(x[idx])
            out_rec.append(rec_image[idx])
            # f_x = netD.forward(x,getFeature=True)[1]
            # f_gx = netD.forward(rec_image,getFeature=True)[1]
            
            rec_diff = torch.abs((rec_image.view(bs, -1) - x.view(bs, -1)))
            rec_score = rec_diff.mean(dim=1)
            # rec_score = ssim_loss(rec_image, x,size_average=False)
            rec_scores.append(rec_score.cpu().numpy())

            enc_diff = torch.abs((rec_code-code))
            #enc_score = enc_diff.mean(dim=1)
            enc_scores.append(enc_diff.mean(dim=1).cpu().numpy())

            # feat_diff = ((f_x - f_gx)**2)
            # feat_score = feat_diff.mean(dim=1) 
            #feat_scores.append(feat_diff.mean(dim=1).cpu().numpy())

            #plt.figure()
            #plt.imshow(x[0,0].cpu())
            #print(outlier_score.cpu().numpy())
            # outlier_score = ssim_loss(rec_image, x, size_average=False)
            y_true.append(label)
            # y_score.append(outlier_score.cpu())

    enc_scores = np.concatenate(enc_scores)
    rec_scores = np.concatenate(rec_scores)
    #feat_scores = np.concatenate(feat_scores)
    rec_scores = (rec_scores - np.min(rec_scores)) / (np.max(rec_scores) - np.min(rec_scores))
    #feat_score = (feat_scores - np.min(feat_scores)) / (np.max(feat_scores) - np.min(feat_scores))
    enc_scores = (enc_scores - np.min(enc_scores)) / (np.max(enc_scores) - np.min(enc_scores))

    y_score = rec_scores + enc_scores #+ feat_scores

    in_real = torch.cat(in_real, dim=0)[:32]
    in_rec = torch.cat(in_rec, dim=0)[:32]
    out_real = torch.cat(out_real, dim=0)[:32]
    out_rec = torch.cat(out_rec, dim=0)[:32]
    save_image(torch.cat((in_real, in_rec), dim=0), 'healthy.jpg', normalize=True)
    save_image(torch.cat((out_real, out_rec), dim=0),
               'unhealthy.jpg', normalize=True)
    # y_score = np.concatenate(y_score)
    y_true = np.concatenate(y_true)
    y_true[y_true != options_c] = -1
    y_true[y_true == options_c] = 1
    print('auc:', metrics.roc_auc_score(y_true, -y_score))
    #plt.figure()
    #plt.hist(y_score[y_true==1], 100, density=True, alpha=0.5, color='blue')
    #plt.hist(y_score[y_true==-1], 100, density=True, alpha=0.5, color='red')
    #plt.show()
    return y_true, y_score

# Test Function

In [25]:
!pip install kornia

In [26]:
from kornia import gaussian_blur2d, median_blur

In [27]:
def test(path):
    options_alpha = 1
    options_c = 0
    netG = model.avgG
    netG.eval()
    netD = model.netD
    netD.eval()
    for p in netD.parameters():
        p.requires_grad = False
    for p in netG.parameters():
        p.requires_grad = False

    netE = Encoder(DIM,NOISE_SIZE).to(device)
    optimizer = optim.Adam(netE.parameters(), lr=1e-4, betas=(0., 0.9),weight_decay=1e-5)
    crit = nn.MSELoss()
    rec_crit = nn.MSELoss()
    
    chkpts = os.listdir('encoder/')
    if not chkpts: 
      print('starting from scratch...')
      start_iter = 1
    else:
      print('loading checkpoint...')
      chkpts = sorted(chkpts, reverse=True, key=lambda x : float(re.split('_|p',x)[1]))
      checkpoint_fpathE = os.path.join('encoder/', chkpts[0])
      print(checkpoint_fpathE)
      checkpointEl = torch.load(checkpoint_fpathE)

      netE.load_state_dict(checkpointEl['state_dict'])
      optimizer.load_state_dict(checkpointEl['optimizer'])

      start_iter = checkpointEl['epoch']
    
    netE.eval()
    _, dataloader = get_dataloaders(train_data_path, path, BATCH_SIZE)
    # crit = nn.MSELoss()
    y_true, y_score = [], []
    in_real, out_real, in_rec, out_rec = [], [], [], []
    
    with torch.no_grad():
        for (x, label) in dataloader:
            bs = x.size(0)
            x = x.to(device)
            # denoise
            x = median_blur(x,(5,5))
            code = netE(x)
            rec_image = netG(code)
            rec_code = netE(rec_image)
            # rec_image = median_blur(rec_image,(5,5))
            d_input = torch.cat((x, rec_image), dim=0)
            idx = (label == options_c)
            in_real.append(x[idx])
            in_rec.append(rec_image[idx])
            idx = (label != options_c)
            out_real.append(x[idx])
            out_rec.append(rec_image[idx])
            f_x = netD.forward(x,getFeature=True)[1]
            f_gx = netD.forward(rec_image,getFeature=True)[1]

            rec_diff = abs((rec_image.view(bs, -1) - x.view(bs, -1)))
            rec_score = rec_diff.mean(dim=1) 
            rec_score = rec_score + ssim_loss(rec_image, x, size_average=False)
            feat_diff = ((f_x - f_gx)**2)
            feat_score = feat_diff.mean(dim=1) 
            enc_score =((rec_code - code)**2)
            enc_score = enc_score.mean(dim=1)
            outlier_score = rec_score + options_alpha * feat_score
            # outlier_score = rec_score + enc_score
            #plt.figure()
            #plt.imshow(x[0,0].cpu())
            #print(outlier_score.cpu().numpy())
            # outlier_score = ssim_loss(rec_image, x, size_average=False)
            y_true.append(label)
            y_score.append(outlier_score.cpu())
    in_real = torch.cat(in_real, dim=0)[:32]
    in_rec = torch.cat(in_rec, dim=0)[:32]
    out_real = torch.cat(out_real, dim=0)[:32]
    out_rec = torch.cat(out_rec, dim=0)[:32]
    save_image(torch.cat((in_real, in_rec), dim=0), 'test-healthy.jpg', normalize=True)
    save_image(torch.cat((out_real, out_rec), dim=0),
               'test-unhealthy.jpg', normalize=True)
    y_score = np.concatenate(y_score)
    y_true = np.concatenate(y_true)
    y_true[y_true != options_c] = -1
    y_true[y_true == options_c] = 1
    print('auc:', metrics.roc_auc_score(y_true, -y_score))
    plt.figure()
    plt.hist(y_score[y_true==1], 100, density=True, alpha=0.5, color='blue')
    plt.hist(y_score[y_true==-1], 100, density=True, alpha=0.5, color='red')
    plt.show()
    return y_true, y_score

# Run

In [28]:
options_eval = False
options_stage = 2
if not options_eval:
    if options_stage == 1:
        wgan_training()
    elif options_stage == 2:
        train_encoder()
else:
    y_true, y_score = evaluate()

starting from scratch...
1 0.5656719654798508
auc: 0.7506576903476001
2 0.5312511258325784
auc: 0.7525768147126747
3 0.5255871087743835
auc: 0.7490660720543965
4 0.5207254160970288
auc: 0.7430163921380177
5 0.5184206461389069
auc: 0.773708608359363
6 0.5144745157004265
auc: 0.7679604176773698
7 0.5106285374483622
auc: 0.7487370355803209
8 0.5091016926073467
auc: 0.6981373474762902
9 0.5067045814197706
auc: 0.7390358160028038
10 0.5045829396998042
auc: 0.7001284007449997
11 0.502275385730521
auc: 0.6981327562696753
12 0.5009242870439026
auc: 0.6827659877292351
13 0.49863266917939214
auc: 0.6971808460981631
14 0.49716692562021353
auc: 0.685341654640256
15 0.49535241520965295
auc: 0.6837469755426424
16 0.4935259596995376
auc: 0.675256304109283
17 0.492031117729425
auc: 0.6898777667758864
18 0.4908550022200262
auc: 0.6630467553177651
19 0.4893766271108745
auc: 0.6967584550895822
20 0.48815156088276324
auc: 0.6611352829637157
21 0.48709799424756933
auc: 0.694253951881094
22 0.48543714420704

KeyboardInterrupt: ignored

In [ ]:
!rm -r encoder/
!mkdir encoder/

for img in os.listdir("./"):
  if img[-3:] == "jpg":
    os.remove(img)

In [ ]:
test(valid_data_path)